<a href="https://colab.research.google.com/github/bhagyasharma/nlp_commonsenseqa/blob/master/csqa_test_random_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instructions

Upload bert_config.json & pytorch_model.bin files from the 'models' folder into your Google Drive. Once you've mounted your drive (in the 3rd cell of this notebook), set the file path of these 2 files in cell 4. This takes care of the model loading portion.  

To test random input, fill out the Question/Answer form by inputting a common sense question with 3 answer options. Run the remaining cells of the notebook to get your answer prediction by this model. 

####Example of Common Sense Questions:  
1. Why do people read gossip magazines? **A) entertained** B) get information C) improve know how  

2. When a person admits his mistakes, what are they doing? **A) act responsibly** B) learn to swim C) feel relieved  

3. Where can books be read? A) shelf **B) table** C) backpack  

** examples from https://www.tau-nlp.org/commonsenseqa



For more information about this model, see https://github.com/bhagyasharma/nlp_commonsenseqa

# Install Dependencies

In [1]:
%%shell
pip install pytorch-pretrained-bert

    100% |████████████████████████████████| 122kB 4.0MB/s 


In [2]:
import torch
import csv
import os
import sys
import numpy as np
from pytorch_pretrained_bert.modeling import BertForMultipleChoice, BertConfig
from pytorch_pretrained_bert.tokenization import BertTokenizer
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


#Import saved model from Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
''' Set file names here'''
config_file = '/content/gdrive/My Drive/Colab Notebooks/bert_config.json'
model_bin_file = '/content/gdrive/My Drive/Colab Notebooks/pytorch_model.bin'

# Load model and supporting methods/classes

In [0]:
# Load a trained model and config that you have fine-tuned
config = BertConfig(config_file)
model = BertForMultipleChoice(config, num_choices=3)
model.load_state_dict(torch.load(model_bin_file, map_location=torch.device('cpu')))

class CSQAExample(object):
    """A single training/test example for the CSQA dataset."""
    def __init__(self,
                 question,
                 choice_0,
                 choice_1,
                 choice_2,
                 choice_3 = None,
                 choice_4 = None,
                 label = None):
        self.question = question
        self.choices = [
            choice_0,
            choice_1,
            choice_2,
            choice_3,
            choice_4
        ]
        self.choices = list(filter(None.__ne__, self.choices))
        self.label = label

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        l = [
            "question: {}".format(self.question),
            "choice_0: {}".format(self.choices[0]),
            "choice_1: {}".format(self.choices[1]),
            "choice_2: {}".format(self.choices[2]),
            "choice_3: {}".format(self.choices[3]),
            "choice_4: {}".format(self.choices[4])
        ]

        if self.label is not None:
            l.append("label: {}".format(self.label))

        return ", ".join(l)


class InputFeatures(object):
    def __init__(self,
                 choices_features,
                 label

    ):
        self.choices_features = [
            {
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            }
            for _, input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label


def read_csqa_example(question, ac0, ac1, ac2):
    examples = []
    examples.append(
        CSQAExample(
        question = question,

        choice_0 = ac0,
        choice_1 = ac1,
        choice_2 = ac2))

    return examples

def convert_examples_to_features(examples, tokenizer, max_seq_length,
                                 is_training):
    """Loads a data file into a list of `InputBatch`s."""

    # Use this formatting for tokenization
    # Each choice will be tokenized accordingly:
    # - [CLS] question [SEP] choice_1 [SEP]
    # - [CLS] question [SEP] choice_2 [SEP]
    # - [CLS] question [SEP] choice_3 [SEP]
    # The model will output a single value for each input. To get the
    # final decision of the model, we will run a softmax over these 3
    # outputs.
    features = []
    for example_index, example in enumerate(examples):
        context_tokens = tokenizer.tokenize(example.question)

        choices_features = []
        for choice_index, choice in enumerate(example.choices):
            # We create a copy of the context tokens in order to be
            # able to shrink it according to choice_tokens
            context_tokens_choice = context_tokens[:]
            choice_tokens = tokenizer.tokenize(choice)
            # Modifies `context_tokens_choice` and `choice_tokens` in
            # place so that the total length is less than the
            # specified length.  Account for [CLS], [SEP], [SEP] with
            # "- 3"
            
            _truncate_seq_pair(context_tokens_choice, choice_tokens, max_seq_length - 3)

            tokens = ["[CLS]"] + context_tokens_choice + ["[SEP]"] + choice_tokens + ["[SEP]"]
            segment_ids = [0] * (len(context_tokens_choice) + 2) + [1] * (len(choice_tokens) + 1)

            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            input_mask += padding
            segment_ids += padding

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            choices_features.append((tokens, input_ids, input_mask, segment_ids))

        label = example.label
        
        features.append(
            InputFeatures(
                choices_features = choices_features,
                label = label
            )
        )

    return features

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
        return
    if len(tokens_a) > len(tokens_b):
        tokens_a.pop()
    else:
        tokens_b.pop()

def select_field(features, field):
    return [
        [
            choice[field]
            for choice in feature.choices_features
        ]
        for feature in features
    ]

#Add test input

In [0]:
question = "Why do people read gossip magazines?" #@param {type:"string"}
answer_choice_0 = "entertained" #@param {type:"string"}
answer_choice_1 = "get information" #@param {type:"string"}
answer_choice_2 = "improve know how" #@param {type:"string"}


#Get predicted answer

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
device = torch.device("cpu")
n_gpu = torch.cuda.device_count()
torch.manual_seed(42)
if n_gpu > 0:
  torch.cuda.manual_seed_all(42)
  
model.to(device)
if n_gpu > 1:
  model = torch.nn.DataParallel(model)
model.to(device)

eval_examples = read_csqa_example(question, answer_choice_0, answer_choice_1, answer_choice_2)
eval_features = convert_examples_to_features(eval_examples, tokenizer, 128, False)
all_input_ids = torch.tensor(select_field(eval_features, 'input_ids'), dtype=torch.long)
all_input_mask = torch.tensor(select_field(eval_features, 'input_mask'), dtype=torch.long)
all_segment_ids = torch.tensor(select_field(eval_features, 'segment_ids'), dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=16)

model.eval()
eval_accuracy = 0
nb_eval_steps, nb_eval_examples = 0, 0
predictions = []
for input_ids, input_mask, segment_ids in eval_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)

    with torch.no_grad():
        tmp_eval_loss = model(input_ids, segment_ids, input_mask)
        logits = model(input_ids, segment_ids, input_mask)
    
    logits = logits.detach().cpu().numpy()
    outputs = np.argmax(logits, axis=1).tolist()
    predictions.extend(outputs)

answer_choices = [answer_choice_0, answer_choice_1, answer_choice_2]
print('Answer is Choice ' + str(predictions[0]) + ': '+ answer_choices[predictions[0]])


Answer is Choice 0: entertained
